# High-level Keras (Theano) Example

In [ ]:
# Lots of warnings!
# Not sure why Keras creates model with float64?

In [1]:
%%writefile ~/.theanorc
[global]
device = cuda0
force_device= True
floatX = float32
warn_float64 = warn

Overwriting /home/iliauk/.theanorc


In [2]:
import os
import sys
import numpy as np
os.environ['KERAS_BACKEND'] = "theano"
import theano
import keras as K
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from common.params import *
from common.utils import *

Using cuDNN version 6021 on context None
Mapped name None to device cuda0: Tesla P100-PCIE-16GB (BC4B:00:00.0)
Using Theano backend.


In [3]:
# Force one-gpu
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [4]:
# Performance Improvement
# 1. Make sure channels-first (not last)
K.backend.set_image_data_format('channels_first')
# 2. CuDNN auto-tune
theano.config.dnn.conv.algo_fwd = "time_once"
theano.config.dnn.conv.algo_bwd_filter = "time_once"
theano.config.dnn.conv.algo_bwd_data = "time_once"

In [5]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Keras: ", K.__version__)
print("Numpy: ", np.__version__)
print("Theano: ", theano.__version__)
print(K.backend.backend())
print(K.backend.image_data_format())
print("GPU: ", get_gpu_name())
print(get_cuda_version())
print("CuDNN Version ", get_cudnn_version())

OS:  linux
Python:  3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Keras:  2.1.4
Numpy:  1.14.1
Theano:  1.0.1
theano
channels_first
GPU:  ['Tesla P100-PCIE-16GB', 'Tesla P100-PCIE-16GB']
CUDA Version 8.0.61
CuDNN Version  6.0.21


In [6]:
def create_symbol(n_classes=N_CLASSES):
    model = Sequential()
    
    model.add(Conv2D(50, kernel_size=(3, 3), padding='same', activation='relu',
                     input_shape=(3, 32, 32)))
    model.add(Conv2D(50, kernel_size=(3, 3), padding='same', activation='relu'))    
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(100, kernel_size=(3, 3), padding='same', activation='relu'))
    model.add(Conv2D(100, kernel_size=(3, 3), padding='same', activation='relu'))    
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.25))
        
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(n_classes, activation='softmax'))
    return model

In [7]:
def init_model(m, lr=LR, momentum=MOMENTUM):
    m.compile(
        loss = "categorical_crossentropy",
        optimizer = K.optimizers.SGD(lr, momentum),
        metrics = ['accuracy'])
    return m

In [8]:
%%time
# Data into format for library
x_train, x_test, y_train, y_test = cifar_for_library(channel_first=True, one_hot=True)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Preparing train set...
Preparing test set...
(50000, 3, 32, 32) (10000, 3, 32, 32) (50000, 10) (10000, 10)
float32 float32 int32 int32
CPU times: user 650 ms, sys: 604 ms, total: 1.25 s
Wall time: 1.25 s


In [9]:
%%time
# Load symbol
sym = create_symbol()

/anaconda/envs/py35/lib/python3.5/site-packages/theano/compile/function_module.py:1519: UserWarning: You are creating a TensorVariable with float64 dtype. You requested an action via the Theano flag warn_float64={ignore,warn,raise,pdb}.
  optimizer_profile = optimizer(fgraph)
/anaconda/envs/py35/lib/python3.5/site-packages/theano/compile/function_module.py:1519: UserWarning: You are creating a TensorVariable with float64 dtype. You requested an action via the Theano flag warn_float64={ignore,warn,raise,pdb}.
  optimizer_profile = optimizer(fgraph)
/anaconda/envs/py35/lib/python3.5/site-packages/theano/compile/function_module.py:1519: UserWarning: You are creating a TensorVariable with float64 dtype. You requested an action via the Theano flag warn_float64={ignore,warn,raise,pdb}.
  optimizer_profile = optimizer(fgraph)


CPU times: user 1.16 s, sys: 163 ms, total: 1.33 s
Wall time: 3.29 s


/anaconda/envs/py35/lib/python3.5/site-packages/theano/compile/function_module.py:1519: UserWarning: You are creating a TensorVariable with float64 dtype. You requested an action via the Theano flag warn_float64={ignore,warn,raise,pdb}.
  optimizer_profile = optimizer(fgraph)
/anaconda/envs/py35/lib/python3.5/site-packages/theano/compile/function_module.py:1519: UserWarning: You are creating a TensorVariable with float64 dtype. You requested an action via the Theano flag warn_float64={ignore,warn,raise,pdb}.
  optimizer_profile = optimizer(fgraph)


In [10]:
%%time
# Initialise model
model = init_model(sym)

CPU times: user 12.2 ms, sys: 0 ns, total: 12.2 ms
Wall time: 12 ms


In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 50, 32, 32)        1400      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 50, 32, 32)        22550     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 16, 16)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 16, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 100, 16, 16)       45100     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 100, 16, 16)       90100     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 100, 8, 8)         0         
__________

In [12]:
%%time
# Main training loop: 1m33s
model.fit(x_train,
          y_train,
          batch_size=BATCHSIZE,
          epochs=EPOCHS,
          verbose=1)

Epoch 1/10
50000/50000 [==============================] - 8s 163us/step - loss: 1.8260 - acc: 0.3317
Epoch 2/10
50000/50000 [==============================] - 8s 158us/step - loss: 1.3923 - acc: 0.4949
Epoch 3/10
50000/50000 [==============================] - 9s 181us/step - loss: 1.1576 - acc: 0.5866
Epoch 4/10
50000/50000 [==============================] - 8s 155us/step - loss: 1.0030 - acc: 0.6450
Epoch 5/10
50000/50000 [==============================] - 8s 153us/step - loss: 0.8973 - acc: 0.6837
Epoch 6/10
50000/50000 [==============================] - 8s 155us/step - loss: 0.8053 - acc: 0.7170
Epoch 7/10
50000/50000 [==============================] - 9s 181us/step - loss: 0.7338 - acc: 0.7404
Epoch 8/10
50000/50000 [==============================] - 8s 154us/step - loss: 0.6722 - acc: 0.7627
Epoch 9/10
50000/50000 [==============================] - 8s 160us/step - loss: 0.6091 - acc: 0.7848
Epoch 10/10
50000/50000 [==============================] - 8s 160us/step - loss: 0.5626 - a

In [13]:
%%time
# Main evaluation loop: 2.47s
y_guess = model.predict(x_test, batch_size=BATCHSIZE)
y_guess = np.argmax(y_guess, axis=-1)
y_truth = np.argmax(y_test, axis=-1)

CPU times: user 886 ms, sys: 191 ms, total: 1.08 s
Wall time: 2.47 s


In [14]:
print("Accuracy: ", 1.*sum(y_guess == y_truth)/len(y_guess))

Accuracy:  0.7525
